# load data

In [ ]:
import sqlite3
import pandas as pd
import re
from urllib.parse import urlparse
import requests

In [ ]:
from datetime import date
from datetime import timedelta

## load tweets older then two weeks

In [ ]:
def load_tweets(db_path, days):
    time_delta = date.today() - timedelta(days=days)
    cnx = sqlite3.connect(db_path)
    query = f"SELECT id,user, full_text, created_at, lang, quoted_status, in_reply_to_status_id FROM tweets WHERE created_at < '{str(time_delta)}'"
    #TODO add restrain, to remove tweets I liked, but for that I need to setup another cron job too.
    df = pd.read_sql_query(
        query,
        cnx,
    )
    return df


In [ ]:
df_tweets = load_tweets("../home.db", days=21)

# utils

In [ ]:
for col in df_tweets.columns:
    if df_tweets[col].dtype == "float64":
        df_tweets[col] = df_tweets[col].fillna(0).astype(np.int64)

In [ ]:
df_tweets.tail()

In [ ]:
def find_url(tweet):
    return re.findall(r"http\S+", tweet)


def clean_links(tweet):
    tweet = re.sub(r"bit.ly/\S+", "", tweet)
    tweet = re.sub(r"t.co/\S+", "", tweet)
    tweet = re.sub(r"buff.ly/\S+", "", tweet)
    tweet = re.sub(r"twitter.com/\S+", "", tweet)
    return tweet

def expand_link(url):
    if url in ["https://", "http://"]:
        return url
    else:
        try:
            session = requests.Session()
            resp = session.head(url, allow_redirects=True)
            long_url = resp.url
            return long_url
        except:
            return url

def get_domain(url):
    domain = urlparse(url).netloc
    dot_split = domain.split(".")
    if len(dot_split) > 2:
        return ".".join(dot_split[1:])
    else:
        return domain


def remove_empty_str(l):
    for i in l:
        if len(i) == 0:
            l.remove(i)
    return l

# searching for not-expanded news urls

In [ ]:
dfz = df_tweets[df_tweets.id == 1339612008182919168]
dfz["urls"] = dfz["full_text"].apply(find_url)
dfz["urls"] = dfz.urls.apply(lambda x: [clean_links(d) for d in x])
dfz["domains"] = dfz.urls.apply(lambda x: [get_domain(d) for d in x])
dfz

In [ ]:
dfz = df_tweets.copy()
dfz["urls"] = dfz["full_text"].apply(find_url)
dfz["urls"] = dfz.urls.apply(lambda x: [clean_links(d) for d in x])
dfz["domains"] = dfz.urls.apply(lambda x: [get_domain(d) for d in x])

In [ ]:
dfz["urls_exp"] = dfz.urls.apply(lambda x: [expand_link(d) for d in x])

In [ ]:
dfz["domains_exp"] = dfz.urls_exp.apply(lambda x: [get_domain(d) for d in x])

In [ ]:
dfz

# function for df

In [ ]:
def find_news(df, news_domains_list):

    df["urls"] = df["full_text"].apply(find_url)
    df["urls"] = df.urls.apply(lambda x: [clean_links(d) for d in x])
    df["domains"] = df.urls.apply(lambda x: [get_domain(d)
    543  for d in x])
    df["domains"] = df.domains.apply(remove_empty_str)
    df.drop(["urls"], axis=1, inplace=True)

    new_columns_list = []
    max_nr_dom = df.domains.str.len().max()
    for i in range(max_nr_dom):
        new_columns_list.append(f"domain{i+1}")
    df[new_columns_list] = pd.DataFrame(df.domains.tolist())

    for col in new_columns_list:
        df[col] = df[col].isin(news_domains_list)

    df.drop(["domains"], axis=1, inplace=True)

    df["contains_news"] = df[new_columns_list].sum(axis=1)
    df["contains_news"] = df.contains_news.apply(lambda x: x if x == 0 else 1)
    df.drop(new_columns_list, axis=1, inplace=True)

    return df

def drop_contains(df, column_name, word_list):
    for string in word_list:
        df["lower"] = df["full_text"].str.lower()
        df = df[df["lower"].str.contains(string)]
        df.drop(["lower"], axis=1, inplace=True)
    return df

In [ ]:
with open("news_domains.txt", "r") as f:
    news_domains = json.loads(f.read())

In [ ]:
%%time
df_tweets = find_news(df_tweets, news_domains)

In [ ]:
def news_in_qt_rt(df):
    #TODO refractor this
    df["all_news"] = df["contains_news"].copy()

    df_qt = df[["id", "contains_news"]].copy()
    df_qt.columns = ["quoted_status", "qt_news"]
    df = df.merge(df_qt, on="quoted_status", how="left")
    df["qt_news"] = df["qt_news"].fillna(0).astype(np.int64)
    df["all_news"] = df["qt_news"].astype(np.int64) + df["contains_news"].astype(np.int64)

    df_qt = df[["id", "contains_news"]].copy()
    df_qt.columns = ["in_reply_to_status_id", "rt_news"]
    df = df.merge(df_qt, on="in_reply_to_status_id", how="left")
    df["rt_news"] = df["rt_news"].fillna(0).astype(np.int64)
    df["all_news"] = df["rt_news"].astype(np.int64) + df["all_news"].astype(np.int64)
    
    return df


In [ ]:
df = df_tweets.copy()

In [ ]:
df.head()

In [ ]:
for col in ["in_reply_to_status_id"]:
    df[col] = df[col].fillna(0).astype(np.int64)

df = news_in_qt_rt(df)

In [ ]:
df_tweets[df_tweets.id == 1335980504735072256]

In [ ]:
df[df.id == 1335967310843060230]

In [ ]:
# I don't have QT in database

In [ ]:
1335980504735072256

In [ ]:
df.info()

In [ ]:
df["in_reply_to_status_id"] = df["in_reply_to_status_id"].fillna(0).astype(np.int64)

df_qt = df[["id", "contains_news"]].copy()
df_qt.columns = ["in_reply_to_status_id", "rt_news"]
df = df.merge(df_qt, on="in_reply_to_status_id", how="left")
df["rt_news"] = df["rt_news"].fillna(0).astype(np.int64)
df["all_news"] = df["rt_news"].astype(np.int64) + df["all_news"].astype(np.int64)

In [ ]:
df = df.merge(df_qt, on="in_reply_to_status_id", how="left")

In [ ]:
%%time
df_tweets = news_in_qt_rt(df_tweets)

In [ ]:
print(df_tweets.shape)

In [ ]:
df_tweets[df_tweets.id == 1197689942396174336]

# remove seen

In [ ]:
seen_tweets = pd.read_csv("seen.csv")

In [ ]:
seen_tweets.head()

In [ ]:
seen_tweets.info()

In [ ]:
seen_tweets.drop_duplicates(inplace=True)

In [ ]:
df_tweets[df_tweets["id"].isin(seen_tweets["tweet_id"].tolist())].shape

In [ ]:
%%time
print(df_tweets.shape)
df_tweets = df_tweets[~df_tweets["id"].isin(seen_tweets["tweet_id"].tolist())]

In [ ]:
# fiter out non english tweets

In [ ]:
df_tweets.shape

In [ ]:
df_tweets = df_tweets[df_tweets["lang"] == "en"]
df_tweets.shape

# suffle them and filter out news

In [ ]:
df_tweets[df_tweets["contains_news"] == 0].shape

In [ ]:
def drop_contains(df, column_name, str_list, lower=True):
    for string in str_list:
        if lower:
            df["filter"] = df[column_name].str.lower().copy()
        if not lower:
            df["filter"] = df[column_name].copy()
        df = df_tweets[~df["filter"].str.contains(string)]
        df = df.drop(["filter"], axis=1).copy()
    return df

In [ ]:
df_tweets.shape

In [ ]:
🍿

In [ ]:
def show_contains(df, column_name, str_list, lower=True):
    for string in str_list:
        if lower:
            df["filter"] = df[column_name].str.lower().copy()
        if not lower:
            df["filter"] = df[column_name].copy()
        df = df_tweets[df["filter"].str.contains(string)]
        df = df.drop(["filter"], axis=1).copy()
    return df

In [ ]:
str_to_show = ["🍿"]
show_contains(df_tweets, column_name="full_text", str_list = str_to_show, lower=False)

In [ ]:
str_to_drop = ["breaking:"]
df_tweets = drop_contains(df_tweets, column_name="full_text", str_list = str_to_drop)
df_tweets.shape

In [ ]:
str_to_drop = ["GOP"]
df_tweets = drop_contains(df_tweets, column_name="full_text", str_list = str_to_drop, lower=False)
df_tweets.shape

In [ ]:
string = "breaking:"

In [ ]:
df_tweets["lower"] = df_tweets["full_text"].str.lower().copy()
df_tweets = df_tweets[~df_tweets["lower"].str.contains(string)]
df_tweets.drop(["lower"], axis=1, inplace=True)

In [ ]:
df_tweets

In [ ]:
df_tweets["full_text"].str.lower()

In [ ]:
to_custom_news_feed = (
    df_tweets[df_tweets["contains_news"] == 0]
    .sample(frac=1)
    .reset_index(drop=True)[:1000]
)
to_custom_news_feed.shape

In [ ]:
to_custom_news_feed

In [ ]:
# to_custom_news_feed[["id", "user"]].to_csv("batch_to_add.csv")